# Data Skeptic Release Automation

* Final asset
* Upload to libsyn
* Enter libsyn details
* Schedule in Lisbyn
* Run update-content.ipynb
* verify site
* facebook post
* transcription

* remove ad (mike north; fiver)
* upload to special place
* regenerate member feed


In [1]:
import pandas as pd
import numpy as np
import json
import boto3
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
from mutagen.mp3 import MP3

In [2]:
config = json.load(open('../config.json', 'r'))

In [4]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [5]:
from elasticsearch import Elasticsearch
config = json.load(open('../config.json', 'r'))
es_user = config['elasticsearch']['username']
es_password = config['elasticsearch']['password']
es_host = config['elasticsearch']['host']
elastic_search_conn = Elasticsearch(es_host, http_auth=(es_user, es_password))

In [3]:
config['elasticsearch']

{'username': 'elastic',
 'password': 'VaTREcfFix5D20W4TgJUCNGD',
 'host': 'https://085e42d20ff5491fbaa010f11fbdfd6f.us-east-1.aws.found.io:9243'}

In [6]:
index_name = 'search_v1'
aws_access_key_id=config['aws']['key']
aws_secret_access_key=config['aws']['secret']

bucketname='dataskeptic.com'
api = 'https://4sevcujref.execute-api.us-east-1.amazonaws.com/prod'
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [7]:

def update_elastic_search(api, elastic_search_conn, s3, index_name, bucketname, row):
    obj = s3.Object(bucketname, src_file)
    html = obj.get()['Body'].read().decode('utf-8')
    blog_id = int(row['blog_id'])
    doc = {
        'blog_id'      : blog_id,
        'title'        : row['title'],
        'prettyname'   : row['prettyname'],
        'guid'         : row['guid'],
        'author'       : row['author'],
        'abstract'     : row['abstract'],
        'date_created' : row['date_created'],
        'publish_date' : row['publish_date'],
        'content'      : html
    }
    print("Going to update " + src_file + " in elastic search")
    result2 = elastic_search_conn.index(index_name, 'blog', doc, id=blog_id)
    if result2['_shards']['failed']==0:
        url = api + "/blog/index/update"
        data = {'blog_id': blog_id}
        r = requests.post(url, json.dumps(data))
        print(r.content)
        return r.status_code == 200
    else:
        print(len(html))
        print(result2)
        return False


In [10]:
import indexv1

In [11]:
index_name = 'search_v1'
return_value = indexv1.update_elastic_search(api, elastic_search_conn, s3, index_name, bucketname)
print(return_value)

https://4sevcujref.execute-api.us-east-1.amazonaws.com/prod/blog/index/update_to_do
blogs: (1, 13)
dataskeptic.com episodes/2019/ncaa-predictions-on-spark.html
Going to update NCAA Predictions on Spark
{'fieldCount': 0, 'affectedRows': 1, 'insertId': 0, 'serverStatus': 2, 'warningCount': 0, 'message': '(Rows matched: 1  Changed: 1  Warnings: 0', 'protocol41': True, 'changedRows': 1}
{'errors': 0, 'successes': 1}


In [10]:
#album_art = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg'

In [12]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [13]:
fname = 'feed.xml'

In [14]:
if os.path.isfile(fname):
    os.remove(fname)

In [15]:
url = 'http://dataskeptic.com/feed.rss'
url = 'http://dataskeptic.libsyn.com/rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [16]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [17]:
len(episodes)

268

In [18]:
guid = episodes[0]['guid']['#text']
print(episodes[0]['title'], guid)

Data Infrastructure in the Cloud 599a9361089c432bacf39e3f3f2f0298


In [25]:
r = requests.get('https://dataskeptic.com/api/refresh')
r.content.decode('utf-8')

'{"status":"ok"}'

In [30]:
pd.read_sql("SELECT * from blog ORDER By blog_id desc limit 1", conn)

,blog_id,prettyname,title,author,abstract,date_created,publish_date,last_rendered,src_file,c_hash,guid,member_link,last_indexed
0,447,/episodes/2019/data-infrastructure-in-the-cloud,Data Infrastructure In The Cloud,Kyle,,2019-05-18 20:08:19,2019-05-18 20:08:19,2019-05-18 20:08:19,episodes/2019/data-infrastructure-in-the-cloud.md,,,,None


In [4]:
! aws s3 cp s3://dataskeptic.com/posts.db.parquet .

download: s3://dataskeptic.com/posts.db.parquet to ./posts.db.parquet


In [5]:
df = pd.read_parquet('posts.db.parquet')

In [6]:
df

,author,description,s3key,title
url,,,,
/blog/episodes/2019/simultaneous-translation,kyle@dataskeptic.com,,blog/episodes/2019/simultaneous-translation.md,Simultaneous Translation
/blog/episodes/2019/large-corpora-and-zipfs-law,kyle@dataskeptic.com,,blog/episodes/2019/large-corpora-and-zipfs-law.md,Large Corpora And Zipfs Law
/blog/episodes/2019/the-transformer,kyle@dataskeptic.com,,blog/episodes/2019/the-transformer.md,The Transformer
/blog/episodes/2019/criticism-of-bleu,kyle@dataskeptic.com,,blog/episodes/2019/criticism-of-bleu.md,Criticism Of Bleu
/blog/episodes/2019/text-mining-in-r,kyle@dataskeptic.com,,blog/episodes/2019/text-mining-in-r.md,Text Mining In R
/blog/episodes/2019/aaron,kyle@dataskeptic.com,,blog/episodes/2019/aaron.md,Aaron
/blog/episodes/2019/human-vs-machine-transcription,kyle@dataskeptic.com,,blog/episodes/2019/human-vs-machine-transcript...,Human Vs Machine Transcription
/blog/episodes/2019/data-infrastructure-in-the-cloud,kyle@dataskeptic.com,,blog/episodes/2019/data-infrastructure-in-the-...,Data Infrastructure In The Cloud
/blog/episodes/2019/---machine-bias,kyle@dataskeptic.com,,blog/episodes/2019/___machine-bias.md,Machine Bias


In [31]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
ORDER BY blog_id desc
LIMIT 2
"""
df = pd.read_sql(q, conn)

In [32]:
if df.iloc[0]['guid'] == '':
    print("ERROR: missing GUID!")
    print("Going to use guid=" + guid)
    if not(df.iloc[0]['guid'] in emap):
        q2 = "update blog set guid='{guid}' where blog_id={blog_id}".format(guid=guid, blog_id = df.iloc[0]['blog_id'])
        print(q2)
        conn.execute(q2)

df = pd.read_sql(q, conn)
print(df.shape[0], 'result:', df.iloc[0]['title'], df.iloc[0]['guid'])

ERROR: missing GUID!
Going to use guid=599a9361089c432bacf39e3f3f2f0298
update blog set guid='599a9361089c432bacf39e3f3f2f0298' where blog_id=447
2 result: Data Infrastructure In The Cloud 599a9361089c432bacf39e3f3f2f0298


# should actually be based on whether or not the pretty name is the destination / show notes page of the episode

In [31]:
#blog_id=404
#q = "update blog set guid='{guid}' where blog_id={blog_id}".format(guid=guid, blog_id=blog_id)
#print(q)
#conn.execute(q)

In [33]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [34]:
import sys

if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    # Not Python 3 - today, it is most likely to be Python 2
    # But note that this might need an update when Python 4
    # might be around one day
    from urllib import urlopen


In [35]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    if guid in emap:
        print("Adding")
        dest = emap[guid]
        response = urlopen(dest)
        data = response.read()
        fname = 'temp.mp3'
        f = open(fname, 'wb')
        f.write(data)
        f.close()
        audio = MP3(fname)
        duration = int(audio.info.length)
        q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
        r1 = conn.execute(q1)
        #dest = album_art
        #q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
        #r2 = conn.execute(q2)

Adding
Adding


In [27]:
#conn.execute("UPDATE blog SET guid='8ed692d61b7e49a3a587800b57fda923' WHERE blog_id=409")

In [36]:
pd.read_sql("select blog_id, prettyname, title, publish_date, guid from blog order by blog_id desc limit 10", conn)

,blog_id,prettyname,title,publish_date,guid
0,447,/episodes/2019/data-infrastructure-in-the-cloud,Data Infrastructure In The Cloud,2019-05-18 20:08:19,599a9361089c432bacf39e3f3f2f0298
1,446,/episodes/2019/ncaa-predictions-on-spark,NCAA Predictions on Spark,2019-05-11 18:14:13,34cce9347f624443b82dbc261d33a695
2,444,/episodes/2019/the-transformer,The Transformer,2019-05-06 22:03:07,68595f430c3240dfa3a38019bc6a3ffb
3,443,/episodes/2019/mapping-dialects-with-twitter-data,Mapping Dialects with Twitter Data,2019-04-26 00:00:00,b589437cfd944a64bbf2f0edf493d36b
4,442,/episodes/2019/sentiment-analysis,Sentiment Analysis,2019-04-19 00:00:00,8b426b91ef564a9f83657ffb7606d89f
5,441,/episodes/2019/attention,Attention,2019-04-12 00:00:00,4eb829e293dd454cb5e8fe42b65800c4
6,440,/episodes/2019/cross-lingual-short-text-matchi...,Cross-lingual Short-text Matching with Deep Le...,2019-04-05 00:00:00,a9fcf4a476e64ba2a08e4296d316f2af
7,439,/episodes/2019/elmo,ELMo,2019-03-29 00:00:00,707cf41bc7cb43f8a897b6c6ac179481
8,438,/episodes/2019/bleu,BLEU,2019-03-22 00:00:00,264d346c178d45548ee791dd98802bbf
9,437,/episodes/2019/simultaneous-translation,Simultaneous Translation,2019-03-15 00:00:00,264d346c178d45548ee791dd98802bbf


In [25]:
#conn.execute("update blog set abstract='The scale and frequency with which information can be distributed on social media makes the problem of fake news a rapidly metastasizing issue. To do any content filtering or labeling demands an algorithmic solution.  \nIn today''s episode, Kyle interviews Kai Shu and Mike Tamir about their independent work exploring the use of machine learning to detect fake news.' where blog_id=402")

# Bonus feed stuff

In [37]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,798,436,http://traffic.libsyn.com/dataskeptic/human-vs...,mp3,Human vs Machine Transcription,Machine transcription (the process of translat...,2019-03-09 03:10:51,NaN,1962.0
1,797,432,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Marc-Alexandre Côté,Marc-Alexandre Côté has studied at Université ...,2019-03-03 03:39:46,-1.0,NaN
2,796,435,http://traffic.libsyn.com/dataskeptic/seq2seq....,mp3,seq2seq Models,A sequence to sequence (or seq2seq) model is n...,2019-03-02 01:54:26,NaN,1300.0
3,795,434,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Julia Silge,Julia Silge is a Data Scientist at Stack Overf...,2019-02-22 23:40:56,-1.0,NaN
4,794,434,http://traffic.libsyn.com/dataskeptic/text-min...,mp3,Text Mining in R,Kyle interviews Julia Silge about her path int...,2019-02-22 16:19:07,NaN,1227.0
5,793,433,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Rasmus Berg Palm,Rasmus Berg Palm finished his Bachelor and Mas...,2019-02-16 06:42:34,-1.0,NaN
6,791,433,http://traffic.libsyn.com/dataskeptic/recurren...,mp3,Recurrent Relational Networks,One of the most challenging NLP tasks is natur...,2019-02-15 16:00:58,NaN,1152.0
7,790,432,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Wendy Tay,Wendy Tay is a Program Manager at Microsoft Re...,2019-02-09 00:41:48,-1.0,NaN
8,789,432,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Kevin Patel,Kevin Patel is a PhD student at the Center for...,2019-02-08 23:57:45,-1.0,NaN
9,785,432,http://traffic.libsyn.com/dataskeptic/text-wor...,mp3,Text World and Word Embedding Lower Bounds,"In the first half of this episode, Kyle speaks...",2019-02-08 16:42:57,NaN,2346.0


In [39]:
conn.execute("""INSERT INTO related_content (blog_id, dest, type, title, body, blog_id2) VALUES (\n             436, 'https://s3.amazonaws.com/dataskeptic.com/guests/2019/andreas-stolcke.jpg', 'person', 'Andreas Stolcke', 'Andreas Stolcke is a Principal Researcher in the Speech and Dialog Research Group at Microsoft, working at the Microsoft Silicon Valley Campus. He has worked on speech recognition; statistical language modeling; recognition of speaker,  dialect and emotion from speech; machine translation; and on software tools for computational linguistics, such as the open-source SRI Language Modeling toolkit. He is also an External Fellow at the International Computer Science Institute (ICSI) in Berkeley, and a Fellow of the Institute of Electrical and Electronics Engineers (IEEE) and of the International Speech Communication Association.', -1)""")

In [41]:
pd.read_sql("SELECT * from dataskeptic.related_content order by content_id desc limit 5", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,799,436,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Andreas Stolcke,Andreas Stolcke is a Principal Researcher in t...,2019-03-10 03:19:30,-1.0,NaN
1,798,436,http://traffic.libsyn.com/dataskeptic/human-vs...,mp3,Human vs Machine Transcription,Machine transcription (the process of translat...,2019-03-09 03:10:51,NaN,1962.0
2,797,432,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Marc-Alexandre Côté,Marc-Alexandre Côté has studied at Université ...,2019-03-03 03:39:46,-1.0,NaN
3,796,435,http://traffic.libsyn.com/dataskeptic/seq2seq....,mp3,seq2seq Models,A sequence to sequence (or seq2seq) model is n...,2019-03-02 01:54:26,NaN,1300.0
4,795,434,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Julia Silge,Julia Silge is a Data Scientist at Stack Overf...,2019-02-22 23:40:56,-1.0,NaN


In [27]:
pd.read_sql("SELECT dest from related_content where content_id=696", conn).iloc[0]['dest']

'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg'

In [16]:
pd.read_sql("SELECT * from bonus_episodes", conn)

,episode_id,title,link,img,desc,pubDate,enclosure_url,guid,last_updated,blog_id
0,1,Introduction,https://dataskeptic.com,https://s3.amazonaws.com/data-skeptic-bonus-fe...,So what's the deal with the bonus feed? How d...,2017-05-01 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,0b79795d3efc95b9976c7c5b933afce2,2017-05-08 01:24:59,NaN
1,2,"Jill Darling, Survey Director for the Understa...",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Featured recently on Data Skeptic, this is the...",2017-05-02 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,7453e2daa86d4e028125f4ee74be97c5,2018-04-01 01:06:14,252.0
2,3,Easily Fooling Deep Neural Networks,https://dataskeptic.com/blog/episodes/2015/eas...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This episode is from the Data Skeptic archives...,2017-05-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,67c7eec88871451c9603d3b361b51e49,2017-05-12 23:42:28,NaN
3,4,"Ernie Tedeschi, Economics, on Poll Re-Weighting",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,Our recent episode Opinion Polling for Preside...,2017-05-07 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,4d67d8112ac54a85bea403d842937d37,2018-04-01 01:06:19,254.0
4,5,"David Carmona, GM of AI at Microsoft",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of AI, cognitive services, and brin...",2017-06-15 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,59105f0a4ee011e7be430e0f2d4e9582,2018-04-01 01:05:43,267.0
5,6,"Rohan Kumar, GM for the Database Systems Group...",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of database as a service, database ...",2017-06-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5bb2dacb4ee011e7be430e0f2d4e9582,2018-04-01 01:05:42,266.0
6,7,Prophecy Fulfillment,https://dataskeptic.com/blog/bonus/prophecy-fu...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,A short recording made in Vietnam,2017-09-12 03:46:50,https://s3.amazonaws.com/data-skeptic-bonus-fe...,02a58e0c976d11e7999d0e0f2d4e9582,2018-04-01 01:03:30,288.0
7,8,Deploying Machine Learning to Production with ...,https://dataskeptic.com/blog/bonus/2018/deploy...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"In this episode, I discuss the process with To...",2018-01-29 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5d2718172a354a85beac03d84e93ffab,2018-04-01 01:03:29,333.0
8,10,AI at Microsoft,https://dataskeptic.com/blog/bonus/2018/ai-at-...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,In this episode I catch up with Paige Bailey a...,2018-03-27 14:38:12,https://s3.amazonaws.com/data-skeptic-bonus-fe...,234518172a654affff6c03684e6332f2,2018-04-01 01:03:26,357.0
9,11,Charlie Maloney - Journalist and Loebner Prize...,https://dataskeptic.com/blog/bonus/2018/charli...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This is the uncut version of my interview with...,2018-04-13 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,8172ef3ffa24a8abea714335c07d85da,2018-04-13 21:25:05,364.0


In [13]:
pd.read_sql("""
SELECT t2.guid, t2.title, t1.content_id, t1.blog_id, t1.duration, t3.guid
from related_content t1
JOIN blog t2
 on t1.blog_id = t2.blog_id
LEFT JOIN bonus_episodes t3
 on t1.dest = t3.enclosure_url
WHERE t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""", conn)

,guid,title,content_id,blog_id,duration,guid


In [11]:
conn.execute("""
UPDATE related_content t1, blog t2, bonus_episodes t3
SET t2.guid = t3.guid
WHERE t1.blog_id = t2.blog_id
 AND t1.dest = t3.enclosure_url
AND t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""")

In [14]:
pd.read_sql("SELECT * from blog order by blog_id desc limit 10", conn)

,blog_id,prettyname,title,author,abstract,date_created,publish_date,last_rendered,src_file,c_hash,guid,member_link,last_indexed
0,398,/bonus/2018/robert-sheaffer,Robert Sheaffer,Kyle,"In this episode, Kyle interviews skeptical aut...",2018-07-24 02:31:17,2018-07-23 00:00:00,2018-07-24 02:31:17,bonus/2018/robert-shaeffer.htm,8bac26fc5ee56bf4ff7b94f595df137e,123ba8a724353a81be1ac03d84e93ff5a,,2018-07-24 02:35:02
1,397,/episodes/2018/the-spread-of-fake-news,The Spread of Fake News,Kyle,How does fake news get spread online? Its not ...,2018-07-20 05:54:38,2018-07-20 17:27:17,2018-07-20 16:50:57,episodes/2018/the-spread-of-fake-news.htm,2acc9b4c75f876e2b55aba9b8190e945,d089f82bbe174f1b99d711b28f5c9ebb,,2018-07-20 16:55:02
2,396,/episodes/2018/fake-news,Fake News,Kyle,"This episode kicks off our new theme of ""Fake ...",2018-07-13 02:51:28,2018-07-13 15:20:54,2018-07-13 03:56:16,episodes/2018/fake-news.htm,575fe7c2decbc7d54a02b09251519fec,5b239f1adacc4d51b265d2bcaf572c31,,2018-07-13 04:00:03
3,395,/episodes/2018/dev-ops-for-data-science,Dev Ops for Data Science,Kyle,We revisit the 2018 Microsoft Build in this ep...,2018-07-11 20:10:18,2018-07-11 20:26:06,2018-07-11 20:22:36,episodes/2018/dev-ops-for-data-science.htm,22edb694572bf5d72201521d3f8da41b,db35a82146574f52ab5e8650d1bb9f43,,2018-07-11 20:25:02
4,394,/episodes/2018/first-order-logic,First Order Logic,Kyle,Logic is a fundamental of mathematical systems...,2018-07-06 04:38:08,2018-07-06 15:23:39,2018-07-06 21:32:36,episodes/2018/first-order-logic.htm,39a007daafc687da076f00cdab6f4d66,df74ad6688b241ce8ff152f0e4b926b0,,2018-07-06 21:35:02
5,393,/episodes/2018/blind-spots-in-reinforcement-le...,Discovering Blind Spots in Reinforcement Learning,Kyle,An intelligent agent trained in a simulated en...,2018-06-29 05:36:12,2018-06-29 15:32:15,2018-06-29 05:36:12,episodes/2018/blind-spots-in-reinforcement-lea...,597368225536bc73177fdfad7cb34f57,a8feaf4664e54744b24e9950b9da50a5,,2018-06-29 05:45:02
6,392,/episodes/2018/transfer-learning,Transfer Learning,Kyle,"On a long car ride, Linhda and Kyle record a s...",2018-06-15 06:42:11,2018-06-15 15:23:39,2018-06-15 06:42:11,episodes/2018/transfer-learning.htm,c2cba9ed3e0b2ede1620db126c38f805,8fa5993bd9554b068f33566eeddb325c,,2018-06-15 06:50:02
7,391,/episodes/2018/medical-imaging-training-techni...,Medical Imaging Training Techniques,Kyle,Medical imaging is a highly effective tool use...,2018-06-08 03:06:42,2018-06-08 14:06:04,2018-06-08 06:40:24,episodes/2018/medical-imaging-training-techniq...,fc85d859a2158e2f044a736d9ba7da00,4b89f3615aa94de8b994abee770ac011,,2018-06-08 06:45:03
8,390,/episodes/2018/defending-against-adversarial-a...,Defending Against Adversarial Attacks,Kyle,"In this weeks episode, our host Kyle intervie...",2018-06-07 16:09:07,2018-06-22 15:25:18,2018-06-22 03:38:50,episodes/2018/defending-against-adversarial-at...,6cda3be3d5ace5f7e68fbc15e50cd924,b4f3fa58280246e59fd16759f90905c7,,2018-06-22 05:45:02
9,389,/episodes/2018/kalman-filters,Kalman Filters,Kyle,A Kalman Filter is a technique for taking a se...,2018-06-01 06:57:39,2018-06-01 14:02:38,2018-06-01 07:34:02,episodes/2018/kalman-filters.htm,efa80798e87a741d3f5fbacc57333794,cde6f37bec7b4d4889f4c099c549fd90,,2018-06-01 07:35:02


In [15]:
conn.execute("UPDATE bonus_episodes set blog_id=398 WHERE episode_id=24")

In [18]:
conn.execute("update related_content set duration=29*60 where content_id=700")

In [20]:
conn.execute("""insert into related_content 
             (blog_id, dest, type, title, body)
             values 
             (398, 
             'https://s3.amazonaws.com/dataskeptic.com/guests/2018/robert-sheaffer.jpg'
             , 'person', 'Robert Sheaffer', 'Robert Sheaffer is a freelance writer and skeptical investigator. His work appears in publications such as Skeptical Inquirer and Fate Magazine. He was a founding member of the Center For Inquiries''s UFO Subcommittee. He is the author of five books.')""")

In [25]:
conn.execute("""insert into related_content 
             (blog_id, dest, type, title, body)
             values 
             (398, 
             'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg',
             'homepage-image', '', '')""")

In [17]:
pd.read_sql("select * from related_content order by content_id desc limit 10", conn)

NameError: name 'conn' is not defined

In [22]:
conn.execute("update related_content set dest='https://s3.amazonaws.com/dataskeptic.com/guests/2018/robert-sheaffer.jpg' where content_id=702")